# cat-in-the-dat-ii

# import

In [ ]:
import numpy as np
import pandas as pd
import tqdm
import os
from collections import deque

import codecs
import copy
from sklearn.metrics import confusion_matrix
import sys
from platform import python_version

import shutil
from sklearn.preprocessing import OneHotEncoder
import csv
from platform import python_version
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# environment

In [ ]:
print(python_version())

In [ ]:
sys.getdefaultencoding()

In [ ]:
%autosave 0

In [ ]:
#最大表示列数の指定（ここでは2000列を指定）
pd.set_option('display.max_columns', 2000)

In [ ]:
#最大表示行数の指定（ここでは50行を指定）
pd.set_option('display.max_rows', 50)

# definition

In [ ]:
### customize define

FEATURE_NUM = 90

TEST_SIZE_VAL = 0.2

# 特徴量削減ステップ
SELECTOR_STEP = .05

TARGET_PROB_CLASS_VALUE = '1'


In [ ]:
read_train_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/cat-in-the-dat-ii/input/train.csv"

read_test_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/cat-in-the-dat-ii/input/test.csv"

output_dir_path = r"/Users/masayuki/Documents/data/python_workspace/Kaggle/cat-in-the-dat-ii/output"

output_file_name = "submission.csv"

drop_col_name_list =["nom_5", "nom_6", "nom_7", "nom_8", "nom_9"]

float_conv_col_name_list = ["day", "month"]

float_nan_conv_dic ={
    # name: [min, max]
    "day": [1,28],
    "month": [1,12],
}

id_col_name="id"
target_col_name = "target"

RANDOM_STATE_VAL = 0


# function

In [ ]:
def dispalyDf(readDf, dispFlag, row_num, dispLabel):
    print("========================= {} =========================".format(dispLabel))
    row_num , col_num = np.shape(readDf)
    print("row_num = {}, col_num = {}".format(row_num, col_num))
    if dispFlag:
        display(readDf.head(row_num))
    print("++++++ None Count All++++++")
    print(readDf.isnull().values.sum())
    print("++++++ None Count ++++++")
    print(readDf.isnull().sum())
    print("++++++ Col Type ++++++")
    print(readDf.dtypes)
#     print("++++++ Unique By Col（Noneは含まないので注意） ++++++")
#     for col_name in readDf.columns.tolist():
#         print("{} : {}".format(col_name, readDf[col_name].nunique()))
    

    

In [ ]:
def chkNum(expect_label, expect_num, act_label, act_num):
    if expect_num != act_num:
        raise Exception("Num Error !! expect({}): {}, act({}) : {}"
                        .format(expect_label, expect_num, act_label, act_num))
    

In [ ]:
def standardScalerDf(inputDf):
    row_num, col_num = np.shape(inputDf)
    
    # 平準化
    scaler = StandardScaler()
    val_array = scaler.fit_transform(np.array(inputDf.values.tolist()))
    standardScDf = pd.DataFrame(val_array, dtype=np.float, columns = inputDf.columns.tolist())
    
    sc_row_num, sc_col_num = np.shape(standardScDf)

    # 行数チェック
    chkNum("expect_row_num", row_num, "sc_row_num", sc_row_num)
    # 列数チェック
    chkNum("expect_col_num", col_num, "sc_col_num", sc_col_num)

    
    del val_array
    return standardScDf


In [ ]:
####################################
##### 次元削減(特徴量選択) を使用する
##### ↓特徴量選択(RandomForestClassifier)
####################################
def getSelectorFeature(inputFeatureDf,y_Df,feature_num, selector_step):
    # 特徴量因子の重要度を推定する分類器をRandomForestClassifierに設定
    # 0未満 の場合はNoneに設定する。
    # 1回のstepで削除する次元数は5%ずつとする
    
    print("特徴量選択数 = ",feature_num)
    if(feature_num < 0):
        # デフォルト値
        print("n_features_to_select = None に設定")
        feature_num = None
    elif feature_num > len(inputFeatureDf.columns.tolist()):
        raise Exception("feature_num({}) is Over colnum({})".format(feature_num, len(inputFeatureDf.columns.tolist())))
    
    # 特徴量選択
    # step:特徴量削除の速度。一度の再帰処理により指定ステップ分の特徴量が消滅する。
    selector = RFE(estimator=RandomForestClassifier(n_estimators=10,
                                                    random_state=RANDOM_STATE_VAL),
                   n_features_to_select=feature_num,
                   step=selector_step)
    selector.fit(inputFeatureDf,y_Df.values.ravel())
    #selector.fit(inputFeatureDf, y_Df.iloc[:,0])

    # 削除対象配列(Falseが削除対象)
    selFlagArray = selector.support_
    
    # 現状のカラム名を取得
    columnIndex = inputFeatureDf.columns
    
    # 削除対象カラム名リスト
    drop_column_list = list()
    # 削除対象配列 でループ(i=0,1,2,・・・)
    print(len(selFlagArray))
    for i in range(len(selFlagArray)):
        # Falseの場合に、削除対象リストに追加
        if(selFlagArray[i] == False):
            drop_column_list.append(columnIndex[i])

    # 不要な列削除
    #print("削除対象カラム = " , drop_column_list)
    inputFeatureNewDf = inputFeatureDf.drop(drop_column_list, axis=1)
    

    return inputFeatureNewDf


In [ ]:
# test用のカラムのDataFrame
def getAdjustTestColDf(inputDf, train_col_list):
    not_enough_col_list = set(train_col_list) - set(inputDf.columns.tolist())
    print("# trainに有ってtestにないカラム名のリスト")
    print("{} : {}".format(len(not_enough_col_list), not_enough_col_list))
        
    dataLen = len(inputDf)
        
    retDf = inputDf.join(
        pd.DataFrame({col_name : [0]*dataLen  for col_name  in not_enough_col_list },
                     dtype=np.float, columns=not_enough_col_list)
    ).loc[:, train_col_list]
    
    # カラムリストが一致するかチェック
    if retDf.columns.tolist() != train_col_list:
        raise Exception("col_name_list not Match train and test.")

    # データ行数が一致するかチェック
    chkNum("dataLen", dataLen, "retDf", len(retDf))

    return retDf

In [ ]:
# 前処理関数
def getBeforeProcessDf(inputDf, drop_col_name_list, float_conv_col_name_list, train_Flg, train_target_Df ,
                       test_size_val, random_state_val, feature_num, selector_step, train_col_list):
    # inputをコピーしておく
    convDf = inputDf.copy()
    
    # 列削除
    convDf = convDf.drop(drop_col_name_list, axis=1)
    
    # カラム名リスト保持
    col_list = convDf.columns.tolist()

    # 連続値のカラムの欠損値補完
    float_conv_dic={}
    for col_name in float_conv_col_name_list:
        # floatにカラム型変換
        convDf[col_name] = convDf[col_name].astype(np.float)
        
        conv_list = \
        [(np.random.randint(
            float_nan_conv_dic[col_name][0],
            float_nan_conv_dic[col_name][1])
          if np.isnan(elem) else elem )
         for elem in convDf[col_name].values.tolist()]
        float_conv_dic[col_name]=conv_list
    ###
    
    float_conv_df = pd.DataFrame(float_conv_dic,dtype=np.float)
    
    col_list_except_float = [col_name for col_name in col_list 
                             if col_name not in float_conv_col_name_list]
    
    del conv_list
    # float以外のDataFrameを抽出
    convDf = convDf.loc[:,col_list_except_float]
    
    # NaNも一つのカテゴリーとしてダミー変数化したい場合は、引数dummy_na=Trueとする。
    convDf = pd.get_dummies(convDf, dummy_na=True, columns=col_list_except_float)
    

    #  全ての特徴量
    allFeatureDf = convDf.join(float_conv_df)

    if train_Flg:
        # trainの場合
        # 標準化
        allFeatureDf =  standardScalerDf(allFeatureDf)

        # ホールドアウト
        trainFeatureDf, testFeatureDf, y_train, y_test = train_test_split(allFeatureDf,     # 特徴量
                                                                          train_target_Df,    # 正解データ
                                                                          test_size=test_size_val ,
                                                                          random_state=random_state_val)
        
        # RandomForest で特徴量選択を行う。
        trainFeatureDf = getSelectorFeature(trainFeatureDf,  y_train, feature_num, selector_step)
        testFeatureDf = testFeatureDf.loc[:,  trainFeatureDf.columns.tolist()]
        
    else:
        # testの場合
    
        trainFeatureDf = None
        # カラムをtrainと合わせる
        testFeatureDf = getAdjustTestColDf(allFeatureDf, train_col_list)
        # 標準化
        testFeatureDf =  standardScalerDf(testFeatureDf)
        
        
        
        y_train = None
        y_test =  None
    
    return trainFeatureDf, testFeatureDf, y_train, y_test

In [ ]:
# モデル入手
def getModel(train_Df, y_train_Df, test_Df,  y_test_Df, target_class_value):
    randF = RandomForestClassifier(n_estimators=10, random_state=RANDOM_STATE_VAL)
    randF.fit(train_Df, y_train_Df.values.ravel())

    print("f1_score = ",f1_score(y_test_Df.values.ravel(), 
                                 randF.predict(test_Df), pos_label = target_class_value))
    print("auc_score = ",roc_auc_score(y_test_Df.values.ravel(), 
                                       randF.predict_proba(test_Df)[:,int(target_class_value)]))
    
    return randF


In [ ]:
# 予測
def getPredictDf(model, test_Df, id_Df , target_class_value, target_col_name):
    return id_Df.join(
        pd.DataFrame(model.predict_proba(test_Df)[:,int(target_class_value)],
                     dtype=np.float, columns=[target_col_name]))


# process

In [ ]:
# ファイル読み込み(train)
train_read_Df = pd.read_csv(read_train_path,
                            header=0,
                            encoding="utf-8",
                            sep='\n',
                            delimiter=',',
                            engine='python',
                            dtype=object)


In [ ]:
# ファイル読み込み(test)
test_read_Df = pd.read_csv(read_test_path,
                           header=0,
                           encoding="utf-8",
                           sep='\n',
                           delimiter=',',
                           engine='python',
                           dtype=object)


In [ ]:
######################################

In [ ]:
trainDf = train_read_Df.copy()
testDf = test_read_Df.copy()

In [ ]:
dispalyDf(trainDf, True, 5,  "trainDf")

In [ ]:
dispalyDf(testDf, True, 5,  "testDf")

In [ ]:
# カラムの差分
set(trainDf.columns.tolist()) - set(testDf.columns.tolist())

In [ ]:
#################

In [ ]:
# train:データのDataFrameと、ターゲットのデータフレームに分ける。
train_target_Df = trainDf.loc[:,[target_col_name]]
trainDf = trainDf.loc[:,[col_name for col_name in trainDf.columns.tolist() if col_name not in [id_col_name,target_col_name]]]

In [ ]:
#dispalyDf(train_target_Df, True, 5,  "train_target_Df")

In [ ]:
#dispalyDf(trainDf, True, 5,  "trainDf")

In [ ]:
# test:idとデータのDataFrameを抽出。
test_id_Df = testDf.loc[:,[id_col_name]]
testDf = testDf.loc[:,[col_name for col_name in testDf.columns.tolist() if col_name != id_col_name]]

In [ ]:
#dispalyDf(test_id_Df, True, 5,  "test_id_Df")

In [ ]:
#dispalyDf(testDf, True, 5,  "testDf")

In [ ]:
#### train のデータ前処理
train_train_Df, train_test_Df, y_train_train, y_train_test = getBeforeProcessDf(
#    trainDf.loc[0:1000,], drop_col_name_list, float_conv_col_name_list, True , train_target_Df.loc[0:1000,],
    trainDf, drop_col_name_list, float_conv_col_name_list, True , train_target_Df,
    TEST_SIZE_VAL, RANDOM_STATE_VAL, FEATURE_NUM, SELECTOR_STEP, None)


In [ ]:
#### test のデータ前処理
_, test_new_Df, _, _ = getBeforeProcessDf(
#    testDf.loc[0:1000,], drop_col_name_list, float_conv_col_name_list, False , None,
    testDf, drop_col_name_list, float_conv_col_name_list, False , None,
    None, None, None, None, train_train_Df.columns.tolist())


In [ ]:
# モデル入手
model = getModel(train_train_Df, y_train_train, train_test_Df,  y_train_test, TARGET_PROB_CLASS_VALUE)

In [ ]:
# 予測
#resultDf = getPredictDf(model, test_new_Df, test_id_Df.loc[0:1000] , TARGET_PROB_CLASS_VALUE, target_col_name)
resultDf = getPredictDf(model, test_new_Df, test_id_Df , TARGET_PROB_CLASS_VALUE, target_col_name)


In [ ]:
dispalyDf(resultDf, True, 5,  "resultDf")

In [ ]:
# ディレクトリなければ作る
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path) 

In [ ]:
ouput_file_path = os.path.join(output_dir_path, output_file_name)
print(ouput_file_path)

In [ ]:
# ファイル出力
resultDf.to_csv(ouput_file_path,
               sep=",",
                header=True,
                index=False,
                mode="w",
                encoding="utf-8",
                line_terminator="\n"
               )


In [ ]:
raise Exception("EEEEEEEEEEEEEEEEEEEEEE")